In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.set_random_seed(2019)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(2,activation = "softmax")   #Adding the Output Layer
])

In [ ]:
model.summary()


In [ ]:
bs=30         #Setting batch size
train_dir = "/content/drive/MyDrive/internship_DL/train"   #Setting training directory
validation_dir = "/content/drive/MyDrive/internship_DL/Valid"   #Setting testing directory
test_dir = "/content/drive/MyDrive/internship_DL/test" 

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(180,180))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))
testdata_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

In [ ]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=150 // bs,
                    epochs=30,
                    validation_steps=50 // bs,
                    verbose=2)

In [ ]:
model.save('/content/drive/MyDrive/internship_DL/Models')

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/internship_DL/Models')

In [ ]:
loaded_model.summary()

# New Section

In [ ]:
import cv2
img = cv2.imread("/content/drive/MyDrive/internship_DL/test/Normal/9000622L.png")

In [ ]:
classes = loaded_model.predict(testdata_generator, batch_size=10)

In [ ]:
testdata_generator =  test_datagen.flow_from_directory(test_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(180,180))

In [ ]:
#predictions = []
#for i in range(0,len(classes)):
    #prediction = np.argmax(classes[0])
    #predictions.append(prediction)

In [ ]:
predictions = []
for i in range(0,len(classes)):
    prediction = np.argmax(classes, axis = 1, out = None)
    predictions.append(prediction)

In [ ]:
testdata_generator.filenames

In [ ]:
classes

In [ ]:
label = model.predict(testdata_generator)

In [ ]:
labelmap = {0:"Normal"  , 1:"Osteoarthritis"}
print("Predicted Class (0 - Normal  , 1-Osteoarthritis ): ", labelmap[np.argmax(label[0])])

In [ ]:
image_names = testdata_generator.filenames

In [ ]:
image_names

In [ ]:
Actual = [i.split("/")[0] for i in image_names]

In [ ]:
Predicted = [labelmap[np.argmax(label[i])] for i in range(0,len(label))]

In [ ]:
import pandas as pd
Prediction_summary = pd.DataFrame(
    {'Image_name': image_names,
     'Actual': Actual,
     'Predicted': Predicted
    })


In [ ]:
correct = 0
wrong = 0

for i in range (0,len(Prediction_summary)):
  if Prediction_summary["Actual"][i] == Prediction_summary["Predicted"][i]:
          correct +=1
  else:
          wrong +=1


In [ ]:
correct